In [1]:
import pandas as pd
import numpy as np
from utils.data import *
from joblib import dump, load
import glob
import os


models = glob.glob("output/models/*.joblib")
print(models)

df = pd.read_csv("ntu-fintech-hw1-new/test.csv", index_col=["index"])

['output/models\\,lp-7169.joblib', 'output/models\\ada-7258.joblib', 'output/models\\ada-7327.joblib', 'output/models\\bagging-7006.joblib', 'output/models\\bagging-7117.joblib', 'output/models\\bagging-7255.joblib', 'output/models\\ex-7006.joblib', 'output/models\\gbc-7145.joblib', 'output/models\\gbc-7345.joblib', 'output/models\\hbc-7288.joblib', 'output/models\\hgb-7140.joblib', 'output/models\\lg-7191.joblib', 'output/models\\lg-7282.joblib', 'output/models\\lgbm-3896.joblib', 'output/models\\lgbm-455.joblib', 'output/models\\lgbm-6719.joblib', 'output/models\\lgbm-7053.joblib', 'output/models\\lgbm-7077.joblib', 'output/models\\lgbm-7211.joblib', 'output/models\\lgbm-7239.joblib', 'output/models\\lgbm-7266.joblib', 'output/models\\lgbm-7270.joblib', 'output/models\\lgbm-7291.joblib', 'output/models\\lgbm-7296.joblib', 'output/models\\lgbm-7304.joblib', 'output/models\\lgbm-7308.joblib', 'output/models\\lgbm-7334.joblib', 'output/models\\lgbm-7343.joblib', 'output/models\\lgbm-735

In [7]:
df.isna().sum()

age                               7
euducation_level                  7
job                               9
marital                          11
have_credit_card                  0
have_housing_loan                 0
have_personal_loan                0
connect_method                    7
previous_connect_month           10
previous_connect_weekday         15
campaign_connect_times           10
after_campaign_connect_day        4
before_campaign_connect_times     5
last_campaign_outcomes            7
employment_rate                   0
consumer_price_index              0
consumer_confidence_index         0
dtype: int64

In [8]:
models.index("output/models\\lgbm-7747.joblib")

83

In [5]:
load("output/models\\lgbm-7553.joblib")


Pipeline(steps=[('sampling', RandomUnderSampler()),
                ('scaler', StandardScaler()),
                ('LGBMClassifier',
                 LGBMClassifier(colsample_bytree=0.9674837979515113,
                                min_child_samples=108, min_child_weight=1e-05,
                                num_leaves=25, reg_alpha=2, reg_lambda=20,
                                subsample=0.23353451549708829))])

In [9]:

# preprocessing
## NaN
#df = fill_nan_with_mean_most(df)
df = df.fillna(df.rolling(2, min_periods=1).sum().mean())
#df = df.fillna(df.rolling(6,min_periods=1).mean())
#df = df.fillna(df.rolling(50,min_periods=3).sum(std=2).mean())

## label
#df = labeling(df)
for feature in df.columns:
    df[feature] = labelencoder.fit_transform(df[feature])

df.head(1)

,age,euducation_level,job,marital,have_credit_card,have_housing_loan,have_personal_loan,connect_method,previous_connect_month,previous_connect_weekday,campaign_connect_times,after_campaign_connect_day,before_campaign_connect_times,last_campaign_outcomes,employment_rate,consumer_price_index,consumer_confidence_index
index,,,,,,,,,,,,,,,,,
20000,13,2,0,2,0,0,0,1,4,1,6,1,0,1,9,23,8


In [10]:
# preprocessing
#df =  preprocessing_only_label(df)
#df =  preprocessing_onehot(df)

test_model = models[83]

X_test = df
print(X_test.shape)
model = load(test_model)
y_test = model.predict(X_test.to_numpy())
ans = pd.DataFrame(X_test.index)
ans["label"] = y_test
ans = ans.set_index("index")
save_name = "output/models/results/"+os.path.basename(test_model).split(".")[0]
ans.to_csv(save_name+".csv")

(19099, 17)


C:\Users\alant\miniconda3\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


'lgbm-6719'

In [19]:
X_test.to_numpy()

array([[30.,  2.,  4., ...,  0.,  1.,  0.],
       [48.,  0.,  5., ...,  1.,  0.,  0.],
       [52.,  2.,  1., ...,  0.,  1.,  0.],
       ...,
       [48.,  0.,  6., ...,  1.,  0.,  0.],
       [31.,  2.,  6., ...,  0.,  1.,  0.],
       [34.,  0.,  1., ...,  0.,  1.,  0.]])

In [12]:
# preprocessing
#df =  preprocessing_only_label(df)
#df =  preprocessing_onehot(df)
X_test =  preprocessing_only_label(df)


for f in models:
    model = load(f)
    save_name = f.replace("\\","\\output\\").replace(".joblib","")
    y_test = model.predict(X_test)
    ans = pd.DataFrame(X_test.index)
    ans["label"] = y_test
    ans = ans.set_index("index")
    ans.to_csv(save_name+".csv")

C:\Users\alant\miniconda3\lib\site-packages\sklearn\base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- connect_method_cellular
- connect_method_telephone
- have_credit_card_no
- have_credit_card_unknown
- have_credit_card_yes
- ...
Feature names seen at fit time, yet now missing:
- connect_method
- have_credit_card
- have_housing_loan
- have_personal_loan
- job
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 32 features, but StandardScaler is expecting 17 features as input.

## test

In [55]:
df = pd.read_csv("ntu-fintech-hw1-new/train.csv", index_col=["index"])
df = df.fillna(df.rolling(2, min_periods=1).sum().mean())
for feature in df.columns:
    df[feature] = labelencoder.fit_transform(df[feature])
X = df.drop(['label'],axis=1)
y = df['label']

In [57]:
model = load("output/models/"+"1637632134"+"/"+"LGBMClassifier-744828"+".joblib") 
model.fit(X, y)

Pipeline(steps=[('sampling', RandomUnderSampler()),
                ('scaler', StandardScaler()),
                ('LGBMClassifier',
                 LGBMClassifier(colsample_bytree=0.5252010149451318,
                                min_child_samples=116, min_child_weight=0.0001,
                                num_leaves=38, reg_alpha=4, reg_lambda=15,
                                subsample=0.15555555555555556))])

In [ ]:
test_model = models[83]

X_test = df
print(X_test.shape)
model = load(test_model)
y_test = model.predict(X_test.to_numpy())
ans = pd.DataFrame(X_test.index)
ans["label"] = y_test
ans = ans.set_index("index")
save_name = "output/models/results/"+os.path.basename(test_model).split(".")[0]
ans.to_csv(save_name+".csv")